In [83]:
import json
import requests
import pandas as pd

In [106]:
with open('data.txt') as json_file:
    data = json.load(json_file)

In [107]:
ids=[]
longitudes=[]
latitudes=[]
improvements=[]
categories=[]

In [108]:
PROJECT_KEY = "api-dev-oa"
API_KEY = "yourtest-outdoora-ctiveapi"
ROOT_URL = "http://www.outdooractive.com/api/"
ROOT_PROJECT_URL = f"{ROOT_URL}project/{PROJECT_KEY}"
headers = {
    "Accept" : "application/json"
}
params = {
    "key" : API_KEY
}

for key in data:
    if('closest' not in data[key]):
        continue
    for k in data[key]['closest']:
        close_soft = data[key]['closest'][k]['foot_distance']
        time_soft = data[key]['closest'][k]['train_time']
    maximum = 0
    valid = True
    for k in data[key]['nearby']:
        try:
            soft = data[key]['nearby'][k]['foot_distance']
            train = data[key]['nearby'][k]['train_time']
            x = close_soft+time_soft - (soft+train)
            if x > maximum:
                maximum = x
        except:
            pass
    if(maximum == 0): continue
    url = ROOT_PROJECT_URL + f"/oois/{key}"
    r = requests.get(url, params=params, headers=headers)
    r = (r.json()['tour'][0]['startingPoint'])
    ids.append(key)
    latitudes.append(r['lat'])
    longitudes.append(r['lon'])
    improvements.append(maximum)
    categories.append('ZH_hikes')
        
        

In [109]:
d = {'id' : ids, 'lon' : longitudes, 'lat': latitudes, 'improvement': improvements, 'category' : categories}
df = pd.DataFrame(data=d)

In [110]:
df = df[df.improvement < 30000]

In [111]:
df.to_csv("ZH_hikes.csv")